In [1]:
#References
#https://github.com/ultralytics/yolov5

# Setup

Clone repo, install dependencies and check PyTorch and GPU.

In [2]:
from google.colab import drive


#This will prompt for authorization.
drive.mount('/content/drive')

#Create a symbolic link, in order no to be able to save weights on drive. Otherwise it gives an error. 
!ln -s /content/drive/My\ Drive /content/mydrive

drive.mount('/content/drive')

import sys
import os
import cv2
import random
import subprocess
import torch
import glob
from IPython.display import Image, clear_output  # to display images


sys.path.insert(0,'/content/mydrive/Codes/YOLOv5-ultralytics/')
base_input_path = '/content/mydrive/Codes/YOLOv5-ultralytics'

%cd /content/mydrive/Codes/YOLOv5-ultralytics

import utils
import models
import data

%pip install -qr "requirements.txt"  # install

display = utils.notebook_init()  # checks

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")


YOLOv5 🚀 2021-12-31 torch 1.10.0+cu111 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.8/225.9 GB disk)
Setup complete. Using torch 1.10.0+cu111 (CPU)


In [3]:
#Config

input_path = '/content/mydrive/Input/YOLO'
dataset_path = "/content/mydrive/Datasets/DFU_RGB/DFUC2020_trainset/train/"
csv_path = "/content/mydrive/Datasets/DFU_RGB/DFUC2020_trainset"
csv_fname = "groundtruth.csv"

In [4]:
createAnnotations = False


def convertToCentroidCoordinates(x_1, y_1, x_2, y_2, image):
  """Converts the representation of the bounding box from the coordinates of the
  top left, and the bottom right point to the coordinates of centroid, 
  and its width and height.
	
  Args:
		x_1: x coordinate of the top left corner of the bbox
		y_1: y coordinate of the top left corner of the bbox
    x_2: x coordinate of the bottom right corner of the bbox
    y_2: y coordinate of the bottom right corner of the bbox
    image: input image

	Returns:
    x_0: x coordinate of the center of the bbox
    y_0: y coordinate of the center of the bbox
    width: width of the bounding box.
    height: height of the bounding box.
	"""

  x_1 = int(x_1)
  x_2 = int(x_2)
  y_1 = int(y_1)
  y_2 = int(y_2)

  #Get image's width and height
  img_w = image.shape[1]
  img_h = image.shape[0]

  #Get bbbox's centroids
  x_0 = (x_2 + x_1)/2
  y_0 = (y_2 + y_1)/2

  #Get bbbox's width and height
  width = x_2 - x_1
  height = y_2 - y_1

  #Convert pixel values to 0-1
  x_0 = x_0 / img_w
  width = width / img_w
  y_0 = y_0 / img_h
  height = height / img_h

  return x_0, y_0, width, height



def create_yolo_annotations(csv_path, filenames, yolo_path):
  """Iterates through each filename and through the csv annotation file 
  of the dataset and creates annotations in yolo format.
  In our case there is only one class, thus it is always 0 (zero-indexed)
	
  Args:
		csv_path: csv annotation file path
		filenames: list containing training images' full paths
    yolo_path: the path to save the annotation .txt files

	Returns:
		saves a .txt annotation file per image containing an object.
    Each row has the following format:
      class x_center y_center width height
	"""
  found_bg = False
  all_imgs = {}

  for fname in filenames:
    firstBBoxIsFound = False
    with open(csv_path,'r') as f:
      for line in f:
        
        line_split = line.strip().split(',')  
        #print(line_split)
        (filename,x1,y1,x2,y2) = line_split		

        if (filename == "filename"):
          continue			

        #Get bboxes for current image
        if (os.path.basename(fname)[:-4] == filename):
          img = cv2.imread(fname)

          x0, y0, w, h = convertToCentroidCoordinates(x1,y1,x2,y2, img)     

          if (firstBBoxIsFound == False): #Create .txt when 1st bbox is found
            firstBBoxIsFound = True
            f_test = open(os.path.join(yolo_path, filename + '.txt'), 'w') #Create
          f_test.write(str(0) + " " + 
                       str(x0) + " " + 
                       str(y0) + " " + 
                       str(w) + " " + 
                       str(h) +'\n') #Append         
     

      if (firstBBoxIsFound == False):
        print("WARNING:No bbox found for image: " + fname)
        continue		


ouput_yolo_path = os.path.join(input_path, "labels")
if not os.path.exists(ouput_yolo_path):
  os.mkdir(ouput_yolo_path)

input_filenames = glob.glob(dataset_path + "*.jpg") #Keeps the train dataset filenames
if (createAnnotations):
  create_yolo_annotations(os.path.join(csv_path, csv_fname), input_filenames, ouput_yolo_path)      

    

In [5]:
createTxtFiles = False


def split_data_set(dataset_dir, output_txt_path):
    """
    Splits the dataset into train and validation sets. 

    Arguments:  
      dataset_dir: The path where the images are located
      output_txt_path

    Returns:
      Saves three .txt files, train.txt, valid.txt and test.txt.      
    """

    f_val = open(os.path.join(output_txt_path, 'valid.txt'), 'w')
    f_train = open(os.path.join(output_txt_path, 'train.txt'), 'w')
    f_test = open(os.path.join(output_txt_path, 'test.txt'), 'w')
    
    #path, dirs, files = next(os.walk(dataset_dir))
    files = glob.glob(dataset_dir + "*.jpg") #Keeps the train dataset filenames
    data_size = len(files)

    ind = 0
    data_test_size = int(0.1 * data_size) #Keep 10% of the dataset for test.
    data_val_size = int((data_size - (0.1 * data_size))*0.1) #The 90% of the training data is for training and the 10% for validation.
    test_array = random.sample(range(data_test_size), k=data_test_size)
    val_array = random.sample(range(data_test_size, data_test_size+data_val_size), k=data_val_size)
    
    for f in files:
        if(f.split(".")[1] == "jpg"):
            ind += 1            
            if ind in val_array:
                f_val.write(f +'\n')
            elif ind in test_array:
                f_test.write(f +'\n')
            else:
                f_train.write(f +'\n')

if (createTxtFiles is True):
  split_data_set(os.path.join(input_path, "images/"), input_path)

# Train


In [48]:
#!python train.py --img 640 --batch 16 --epochs 100 --data ../../Input/YOLO/dataset.yaml --weights yolov5s.pt
!python train.py --img 640 --batch 16 --epochs 150 --data ../../Input/YOLO/dataset.yaml --weights yolov5l.pt

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5l.pt, cfg=, data=../../Input/YOLO/dataset.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=150, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=../../Results/YOLO/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 2021-12-31 torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, l

In [ ]:
pip install wandb

In [61]:
!ls 

data	   hubconf.py	requirements.txt  utils       yolov5s.pt
detect.py  models	setup.cfg	  val.py
export.py  __pycache__	train.py	  yolov5l.pt


# Test


In [7]:
#!python val.py --weights /content/mydrive/Results/YOLO/train/exp/weights/best.pt --img 640 --data ../../Input/YOLO/dataset.yaml --task test --batch-size 1 --single-cls --conf-thres 0.25
!python val.py --weights /content/mydrive/Results/YOLO/train/exp/weights/best.pt --img 640 --data ../../Input/YOLO/dataset.yaml --task test --batch-size 1 --single-cls --conf-thres 0.25 --iou-thres 0.3

LEN: 20
val: data=../../Input/YOLO/dataset.yaml, weights=['/content/mydrive/Results/YOLO/train/exp/weights/best.pt'], batch_size=1, imgsz=640, conf_thres=0.25, iou_thres=0.3, task=test, device=, workers=8, single_cls=True, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=../../Results/YOLO/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 2021-12-31 torch 1.10.0+cu111 CPU

Fusing layers... 
Model Summary: 367 layers, 46108278 parameters, 0 gradients, 107.8 GFLOPs
test: Scanning '/content/mydrive/Input/YOLO/test.cache' images and labels... 4 found, 0 missing, 0 empty, 0 corrupted: 100% 4/4 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95:   0% 0/4 [00:00<?, ?it/s]
IOU: tensor([[0.83246]])
               ulcer          1          1          1          1      0.995      0.697
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95:  25